<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/03-ss3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo 50 JAIIO: SS3
__Título: E-mail mining:__ un enfoque semi-supervisado para la identificación, el etiquetado y la clasificación de consultas académicas

__Autores:__ Juan Manuel Fernandez, Marcelo Errecalde


En esta notebook se presentan los experimentos sobre _feature extraction_ con SS3 para las 50 JAIIO.

## Instalación y Carga de librerías y funciones útiles

### Instalación de librerías

Se instalan las librerías que no están en el entorno de Google Colab

In [1]:
# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install requests
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=4a38846224d9e735a9ae3d81a593220e65824f9848d5c8f0793760d0a390d1be
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


### Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [2]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [3]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos

### Carga de datos

Se carga el dataframe en memoria con el preprocesamiento de los datos:

In [4]:
import warnings
from os import path
warnings.filterwarnings("ignore")

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/'
TRAIN_FILE = 'correos-train-80.csv'
TEST_FILE = 'correos-test-20.csv'

# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE))

etiquetas = get_clases()

# Defino la cantidad de clases a utilizar
CANTIDAD_CLASES = len(etiquetas)

train_df, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

# Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
import pandas as pd
train_df['Consulta'] = pd.Series(preprocesar_correos(train_df['Consulta']))
test_df['Consulta'] = pd.Series(preprocesar_correos(test_df['Consulta']))

# Cambio los integers por las etiquetas
train_df['clase'] = etiquetas[train_df['clase']]
test_df['clase'] = etiquetas[test_df['clase']]

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.clase.unique()), train_df.clase.unique()))

Se inicia descarga de los datasets.

El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 20 clases: ['Inscripción a Cursadas' 'Cambio de Carrera' 'Reincorporación'
 'Ingreso a la Universidad' 'Boleto Universitario'
 'Pedido de Certificados' 'Exámenes' 'Requisitos de Ingreso' 'Cursadas'
 'Situación Académica' 'Vacunas Enfermería' 'Consulta por Legajo'
 'Problemas con la Clave' 'Consulta sobre Título Universitario'
 'Certificados Web' 'Carga de Notas' 'Otras Consultas'
 'Cambio de Comisión' 'Consulta por Equivalencias' 'Datos Personales'].


## SS3

Se instala la librería:

In [5]:
pip install pyss3

     |████████████████████████████████| 2.0MB 3.0MB/s 


Se instancia la técnica de __SS3__ y se ajustan los hiperparámetros:

In [6]:
from sklearn.preprocessing import MinMaxScaler
from pyss3 import SS3
from pyss3.util import Evaluation, span
from pyss3.server import Live_Test

clf = SS3()

clf.train(train_df['Consulta'], train_df['clase'], prep=False)

best_s, best_l, best_p, best_a = Evaluation.grid_search(
  clf, test_df['Consulta'], test_df['clase'],
  s=span(0.2, 0.8, 6),
  l=span(0.1, 2, 6),
  p=span(0.5, 2, 6),
  a=[0, .1, .2],
  prep=False,  # <- do not forget to disable default preprocessing
  tag="grid search (test)"
)

print("Smoothness(s):", best_s)
print("Significance(l):", best_l)
print("Sanction(p):", best_p)
print("Alpha(a):", best_a)

Evaluation.plot()

Grid search: 100%|██████████| 648/648 [01:46<00:00,  6.11it/s]


Smoothness(s): 0.44
Significance(l): 0.1
Sanction(p): 0.5
Alpha(a): 0.0


True

Se genera el modelo en base a los mejores hiperparámetros encontrados:

In [7]:
from sklearn.metrics import accuracy_score

clf.set_hyperparameters(0.44, 0.1, 0.5, 0.0)
y_pred = clf.predict(test_df['Consulta'], prep=False)

accuracy = accuracy_score(y_pred, test_df['clase'])
print("\nAccuracy was:", accuracy)

Classification: 100%|██████████| 200/200 [00:00<00:00, 23798.82it/s]


Accuracy was: 0.715


In [17]:
clf.print_categories_info()



 CATEGORIES:

	             Category               | Index |   Length   | Vocab. Size | Word Max. Fr. | N-gram |
	------------------------------------|-------|------------|-------------|---------------|--------|
	       Boleto Universitario         |   0   |    3454    |     815     |      159      |   1    |
	         Cambio de Carrera          |   1   |    220     |     142     |      12       |   1    |
	        Cambio de Comisión          |   2   |    120     |     80      |       5       |   1    |
	          Carga de Notas            |   3   |    293     |     187     |      13       |   1    |
	         Certificados Web           |   4   |     78     |     63      |       3       |   1    |
	    Consulta por Equivalencias      |   5   |    232     |     151     |       9       |   1    |
	        Consulta por Legajo         |   6   |    174     |     107     |      15       |   1    |
	Consulta sobre Título Universitario |   7   |    206     |     124     |      14       |   1

In [39]:
clf.gv('boleto', 'Boleto Universitario')

0.9987009855589738

In [28]:
clf.print_ngram_info('boleto')


 WORD: boleto (index: 12)

 Category                            |    fr    |  lv   |  sg   |  sn   |  gv   |  cv   |
 ------------------------------------|----------|-------|-------|-------|-------|-------|
 Boleto Universit                    |   159    | 1.000 | 1.000 | 0.999 | 0.999 | 1.000 |
 Cambio de Carrer                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Cambio de Comisi                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Carga de Notas                      |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Certificados Web                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Consulta por Equ                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Consulta por Leg                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Consulta sobre T                    |    0     | 0.000 | 0.000 | 0.688 | 0.000 | 0.000 |
 Cursadas                            |    0     | 0.000 | 0.000 | 0.688 

Se verifica el __Test Live__:

In [8]:
Live_Test.run(clf, test_df['Consulta'], test_df['clase'])

# Referencias
- https://pyss3.readthedocs.io/en/latest/
- https://hub.gke2.mybinder.org/user/sergioburdisso-pyss3-tot89t3p/notebooks/examples/extract_insight.ipynb